In [ ]:
# Creating a key for reproducibility
key = jax.random.PRNGKey(1)

<https://www.tensorflow.org/datasets/api_docs/python/tfds/load>

In [ ]:
# Build your input pipeline
dataset = dataset.shuffle(1024).batch(32).prefetch(
    tf.data.experimental.AUTOTUNE)
for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]

In [ ]:
import numpy as np
# import tensorflow as tf
from pathlib import Path
from mido import MidiFile
import mido

seed = 42
# tf.random.set_seed(seed)
np.random.seed(seed)



def play_audio(midi_data: MidiFile, duration: float=None) -> None:
        
    # Create output port for playing samples
    with mido.open_output(autoreset=True) as outport: 
        if isinstance(duration, (float, int)):
            for msg in midi_data.play():
                if duration <= 0: break
                outport.send(msg)
                duration -= msg.time
        else:
            for msg in midi_data.play():
                outport.send(msg)

    if not outport.closed:
        print("Closing port...")
        outport.close()

def get_tempo(mid):
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'set_tempo':
                return msg.tempo
    return False  # Default tempo if not found


In [ ]:
import pretty_midi


# Initiate data structures
X_train = []
X_test = []
X_validation = []
# X_train = [0] * 897
# X_test = [0] * 129
# X_validation = [0] * 124
# y_train = np.zeros(())
# y_test = np.zeros(())

# Define filename and its path
data_filename = "info.csv"
data_folder_path = Path().cwd() / "data"
data_path = data_folder_path / data_filename

# bpm_stats = {}

# Open file with data information
with data_path.open() as file:
    columns = file.readline()
    train_idx = 0
    test_idx = 0
    validation_idx = 0
    for i, line in enumerate(file):
        # Strip line of new line and split into list
        line = line.strip().split(",")

        bpm = line[4]
        time_signature = line[6]
        # if bpm in bpm_stats.keys():
        #     bpm_stats[bpm] += 1
        # else:
        #     bpm_stats[bpm] = 1
        if time_signature != "4-4":
            continue

        # Check
        # print(line)

        # Attach absolute path to midi & audio files
        midi_data_path = data_folder_path / line[7]
        audio_data_path = data_folder_path / line[8]

        # Create custom data point with all info
        midi_data = MidiFile(midi_data_path)


        features = {i: line[feature_to_column_idx[i]] for i in feature_names}
        data_point = DataPoint(
            midi_data=midi_data,
            audio_file_path=audio_data_path,
            midi_file_path=midi_data_path,
            features=features,
            purpose=line[feature_to_column_idx["split"]]
        )

        # Play the sample
        # play_audio(midi_data, 3)

        if data_point.purpose == "train":
            # X_train[train_idx] = data_point
            # train_idx += 1
            X_train.append(data_point)
        elif data_point.purpose == "test":
            # X_test[test_idx] = data_point
            # test_idx += 1
            X_test.append(data_point)
        elif data_point.purpose == "validation":
            # X_validation[validation_idx] = data_point
            # validation_idx += 1
            X_validation.append(data_point)

        print(len(X_train), len(X_test), len(X_validation))

        # Load the MIDI file
        midi_data = pretty_midi.PrettyMIDI(str(midi_data_path))
        # for inst in midi_data.instruments:
            # print(inst)
        print(midi_data.instruments)
        # print(line.strip().split(","))
        print("Next sample\n")
        if i == 10: print("Breaking"); break

print(len(X_train))
print(len(X_test))
print(len(X_validation))

In [4]:
X_train[0].midi_file_path

WindowsPath('c:/Users/mrmar/Downloads/Neural Networks/Project/NN_Drummer_Beat_Project/data/drummer1/session1/101_dance-disco_120_beat_4-4.mid')

In [5]:
import jax.numpy as jnp

# arange
# linspace
# :list
# ones
# zeros
# identity

test = jnp.identity(2, jnp.int16)
print(test)

seed = 42
key = jax.random.PRNGKey(seed)
print(key)
jax.random.uniform(key)
key, subkey = jax.random.split(key)
print(key, subkey)

[[1 0]
 [0 1]]
[ 0 42]
[2465931498 3679230171] [255383827 267815257]


In [18]:
testing = PrettyMIDI(str(X_train[1].midi_file_path))
testing = filter_to_kick_drum(testing)
testing.instruments[0].notes



-> Filtering everything but kick... DONE


[]